In [1]:
from get_data import *
import numpy as np
language = "English"
# English
# min_req len(vocab) micro_avg_f1 
# 1 23624 0.5667
# 2 11983 0.7720
# 3 8137  0.7468
# 4 6182  0.7242
# 5 4914  0.6960
# 6 4147  0.6715
# Chinese
# min_req len(vocab) micro_avg_f1 
# 1 1793  0.7871
# 2 1410  0.8379
# 3 1229  0.8475
# 4 1100  0.8549
# 5 994   0.8546
# 6 928   0.8535


# 初始概率矩阵 π:只看每sentence开头的label
π = np.array([0.2, 0.4, 0.4])
# 转移概率矩阵 A:统计prev_label,cur_label
A = np.array([[0.5, 0.2, 0.3], [0.3, 0.5, 0]])
# 观测概率矩阵 B:给定label下出现word的概率
B = np.array([])
# 平滑技术
train_data = get_train_data(language)


In [2]:
from collections import Counter,defaultdict
from check import sorted_labels_eng,sorted_labels_chn
sorted_labels = sorted_labels_chn if language == "Chinese" else sorted_labels_eng
π = np.zeros((len(sorted_labels),1))
for sentence in train_data:
    π[sorted_labels.index(sentence[0][1])][0]+=1
A = np.zeros((len(sorted_labels),len(sorted_labels)))
for sentence in train_data:
    for i in range(len(sentence)-1):
        A[sorted_labels.index(sentence[i][1])][sorted_labels.index(sentence[i+1][1])]+=1
sorted_labels


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [3]:
min_freq = 2 if language == "English" else 4
# 994
# 4 1100
# 3 1228
# 2 1409
# 1 1792

# 1 23624
# 2 11983
import os
import torchtext
train_word = [word  for sentence in train_data for word,label in sentence]
print(len(train_word))
print(train_word)
vocab = torchtext.vocab.vocab(Counter(train_word),min_freq=min_freq,specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])
print(len(vocab))
print(vocab["的"])
print(vocab["劵"])
print(vocab.lookup_token(10))
print(vocab.lookup_token(1))
print(vocab['<unk>'])


203621
['Rangarajan', 'said', 'a', 'current', 'account', 'deficit', 'of', 'two', 'percent', 'brought', 'about', 'by', 'a', '16-17', 'percent', 'annual', 'growth', 'in', 'exports', 'and', 'a', '14-15', 'percent', 'rise', 'in', 'imports', 'along', 'with', 'an', 'increase', 'in', 'non-debt', 'flows', 'could', 'lead', 'to', 'a', 'reduction', 'in', 'the', 'debt-service', 'ratio', 'to', 'below', '20', 'percent', 'over', 'the', 'next', 'five', 'years', '.', '4.', 'Luca', 'Cadalora', '(', 'Italy', ')', 'Honda', '1:51.006', 'BANGKOK', '1996-08-26', 'The', 'PUK', 'said', 'the', 'ceasefire', 'was', 'agreed', 'after', 'talks', 'between', 'U.S.', 'Assistant', 'Secretary', 'for', 'Near', 'East', 'Affairs', 'Robert', 'Pelletreau', 'and', 'PUK', 'leader', 'Jalal', 'Talabani', '.', 'Scores', ':', 'England', '326', 'and', '74-0', ';', 'Pakistan', '521-8', 'declared', '.', 'HONG', 'KONG', '1996-08-29', 'Yeltsin', "'s", 'wife', 'has', 'kidney', 'operation', '.', 'SOCCER', '-', 'HONDURAS', 'BEAT', 'CUBA', 

In [22]:
B = np.zeros((len(sorted_labels),len(vocab)))
for sentence in train_data:
    for i in range(len(sentence)):
        B[sorted_labels.index(sentence[i][1])][vocab[sentence[i][0]]]+=1

In [23]:
# 归一化
eplison = 1e-6
row_sums = π.sum(axis=0)
row_sums
π_prob = π/row_sums[0]
# Ai,j表示li转移到lj的概率
A_prob = np.zeros((len(sorted_labels), len(sorted_labels)))
for i in range(len(sorted_labels)):
    j_sum = A[i].sum()+eplison
    A_prob[i, :] = A[i, :]/j_sum


B_prob = np.zeros((len(sorted_labels), len(vocab)))
for i in range(len(sorted_labels)):
    j_sum = B[i].sum()+eplison
    B_prob[i, :] = B[i, :]/j_sum

for i in range(len(sorted_labels)):
    per_sum = 0
    for j in range(len(sorted_labels)):
        per_sum += A_prob[i, j]
    print(per_sum)
for i in range(len(sorted_labels)):
    per_sum = 0
    for j in range(len(vocab)):
        per_sum += B_prob[i, j]
    print(per_sum)


0.9999999999936001
0.9999999998469778
0.9999999997701677
0.9999999998391765
0.9999999997246697
0.9999999998577525
0.9999999990966575
0.9999999997070885
0.9999999990366089
0.9999999999942392
0.9999999998484684
0.9999999997791511
0.999999999841812
0.9999999997300326
0.9999999998599478
0.9999999991356955
0.9999999997091361
0.999999999134203


In [24]:
from check import check


def viterbi_decoder(vocab, sorted_labels, π, A, B, sentence):
    sentence = [vocab[word] for word in sentence]
    prob = [[0 for _ in range(len(sorted_labels))]
            for _ in range(len(sentence))]
    path = [[0 for _ in range(len(sorted_labels))]
            for _ in range(len(sentence))]
    for i in range(len(sentence)):
        for j in range(len(sorted_labels)):
            if i == 0:
                prob[i][j] = π[j]*B_prob[j][sentence[i]]
                path[i][j] = j
            else:
                max_prob = 0
                max_path = 0
                for k in range(len(sorted_labels)):
                    temp_prob = A_prob[k][j] *\
                        B_prob[j][sentence[i]]*prob[i-1][k]
                    if temp_prob > max_prob:
                        max_prob = temp_prob
                        max_path = k
                path[i][j] = max_path
                prob[i][j] = max_prob
    max_prob = 0
    max_path = 0
    for i in range(len(sorted_labels)):
        if prob[-1][i] > max_prob:
            max_prob = prob[-1][i]
            max_path = i
    paths = [max_path]
    for i in range(len(sentence)-1, 0, -1):
        paths.insert(0, path[i][max_path])
        max_path = path[i][max_path]
    result = [sorted_labels[i] for i in paths]
    return result



In [25]:
# import torch

# def viterbi_decoder(π, A, B, emission):
#     batch_size, seq_len, num_classes = emission.size()
#     prob = torch.zeros((len(sentence),len(sorted_labels)))
#     path = torch.zeros((len(sentence),len(sorted_labels)), dtype=torch.long)
#     prob[ :, 0] = π + temp_emission.squeeze(1)
#     for j in range(1,seq_len):
#         temp_emission = emission[j, :].unsqueeze(1)  # shape: (batch_size, 1, num_classes)
        
#         last_score = prob[:, :, j - 1].unsqueeze(2)
#         score_matrix = last_score + A.transpose(0, 1) + temp_emission
#         prob[:, :, j], path[:, :, j] = torch.max(score_matrix, dim=1)

#     # Backtrack to find the best path
#     _, best_last_tag = torch.max(prob[:, :, -1], dim=1)
#     best_path = torch.zeros((batch_size, seq_len), dtype=torch.long)
#     best_path[:, -1] = best_last_tag

#     for j in range(seq_len - 2, -1, -1):
#         best_last_tag = path[torch.arange(batch_size), best_last_tag, j]
#         best_path[:, j] = best_last_tag

#     return best_path



In [26]:

val_data = get_valid_data(language)


def sentence2word(sentence):
    return [word for word, _ in sentence]


viterbi_decoder(vocab, sorted_labels, π_prob, A_prob,
                B_prob, sentence2word(val_data[0]))


['B-ORG', 'O', 'B-ORG', 'O']

In [27]:
pred_path = "example_data/my_path_HMM_{}.txt".format(language)
with open(pred_path, "w") as f:
    for sentence in val_data:
        pred = viterbi_decoder(vocab,sorted_labels,π_prob,A_prob,B_prob,sentence2word(sentence))
        for i in range(len(sentence)):
    
            f.write(sentence[i][0]+' '+pred[i]+'\n')
        f.write('\n')

check(language,"{}/validation.txt".format(language),pred_path)

              precision    recall  f1-score   support

       B-PER     0.9095    0.6330    0.7465      1842
       I-PER     0.8813    0.6985    0.7793      1307
       B-ORG     0.8049    0.6831    0.7390      1341
       I-ORG     0.8510    0.5779    0.6883       751
       B-LOC     0.9086    0.7627    0.8292      1837
       I-LOC     0.8529    0.6770    0.7549       257
      B-MISC     0.9034    0.7711    0.8321       922
      I-MISC     0.8413    0.6127    0.7090       346

   micro avg     0.8779    0.6889    0.7720      8603
   macro avg     0.8691    0.6770    0.7598      8603
weighted avg     0.8785    0.6889    0.7708      8603

